<a href="https://colab.research.google.com/github/ccalvocm/GEE/blob/testFeatureCollection/Flood_Mapping_using_Google_Earth_Engine_and_SAR_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Earth Engine (GEE) and Synthetic Aperture Radar (SAR) Data for Flood Mapping in Nile Basin

This notebook presents the detection of flooded area over Khartoum, Sudan and its surrounding region from the overflow of Nile Rivers and heavy rains by utilizing Sentinel-1 synthetic aperture radar (SAR) data acquired during dry and wet seasons of 2020. According to the [International Federation of Red Cross and Red Crescent Societies's (IFRC) ](https://www.ifrc.org/en/news-and-media/news-stories/africa/sudan/heavy-rains-and-flash-floods-in-sudan-66633/) report, the heavy rains (from middle July to August) and overflow of Nile Rivers (both Blue and White) have caused flooding, destruction of infrastructure, houses and livelihoods, and have displaced thousands of people in multiple locations across the country, primiarly in Khartoum and its surrounding area.

<div align="center">
  <img width="800" height="500" src="https://www.dabangasudan.org/uploads/cache/article_detail_image/uploads/media/5f589cb1db2d9.png">
</div>

## Authenticate to Colab and Cloud

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Authenticate to Earth Engine

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-QAhUW0C1dlelisYtkok_wP4jtIPnW1BIwMn-XBxPK4&tc=QHXn8cd3IlDUElJc5ifXR_Os42FXSedEJNkW1O7TKJ0&cc=M3GIw2Wpzbrs58E0YwJ_XMq-3ymvt1DyWCaKBa4mgZ0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhUk-oyeb344EsAZ9lk9DTzvKKnhMKvRYHTzfTJvOjB_G8L674l2lCk

Successfully saved authorization token.


### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Test the Folium installation

We will use the Folium library for visualization.  Import the library and check the version.

In [2]:
import folium
print(folium.__version__)

0.14.0


In [3]:
## Define Region of Interest (ROI)

roi_Nuble_flood = ee.Geometry.Polygon([
[-72.02068198409884,-36.188943069996746],
[-70.62541831222384,-36.188943069996746],
[-70.62541831222384,-34.91303129593276],
[-72.02068198409884,-34.91303129593276],
[-72.02068198409884,-36.188943069996746]])

## Load Sentinel-1 C-band SAR Ground Range collection (log scale, VV, descending)
collectionVV = ee.ImageCollection('COPERNICUS/S1_GRD')\
.filter(ee.Filter.eq('instrumentMode', 'IW'))\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
.filterMetadata('resolution_meters', 'equals' , 10)\
.filterBounds(roi_Nuble_flood)\
.select('VV')

## Load Sentinel-1 C-band SAR Ground Range collection (log scale, VH, descending)
collectionVH = ee.ImageCollection('COPERNICUS/S1_GRD')\
.filter(ee.Filter.eq('instrumentMode', 'IW'))\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
.filterMetadata('resolution_meters', 'equals' , 10)\
.filterBounds(roi_Nuble_flood)\
.select('VH')

## Filter by season
wet2020VV = collectionVV.filterDate('2023-06-21', '2023-06-28').mosaic()
dry2020VV = collectionVV.filterDate('2023-01-01', '2023-02-28').mosaic()
wet2020VH = collectionVH.filterDate('2023-06-21', '2023-06-28').mosaic()
dry2020VH = collectionVH.filterDate('2023-01-01', '2023-02-28').mosaic()

In [4]:
## Use folium to visualize the imagery.

mapidVVwet = wet2020VV.getMapId({'min': -15, 'max': 0})
mapidVVdry = dry2020VV.getMapId({'min': -15, 'max': 0})
mapidVHwet = wet2020VH.getMapId({'min': -25, 'max': 0})
mapidVHdry = dry2020VH.getMapId({'min': -25, 'max': 0})

map = folium.Map(location=[-71.82400911149269,-36.29123750152623],zoom_start=8)
folium.TileLayer(
    tiles=mapidVVwet['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Wet2020VV',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidVVdry['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Det2020VV',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidVHwet['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Wet2020VH',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidVHdry['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Det2020VH',
  ).add_to(map)

In [5]:
## Apply filter to reduce speckle
SMOOTHING_RADIUS = 50;
wet2020VV_filtered = wet2020VV.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters')
dry2020VV_filtered = dry2020VV.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters')
wet2020VH_filtered = wet2020VH.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters')
dry2020VH_filtered = dry2020VH.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters')

### Comparing speckle filtered and unfiltered images

In [6]:
mapidVVwet_fil = wet2020VV_filtered.getMapId({'min': -15, 'max': 0})
mapidVVdry_fil = dry2020VV_filtered.getMapId({'min': -15, 'max': 0})
mapidVHwet_fil = wet2020VH_filtered.getMapId({'min': -25, 'max': 0})
mapidVHdry_fil = dry2020VH_filtered.getMapId({'min': -25, 'max': 0})

mapcom = folium.Map(location=[-71.82400911149269,-36.29123750152623],zoom_start=8)
folium.TileLayer(
    tiles=mapidVVwet_fil['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Wet2020VV Filtered',
  ).add_to(mapcom)
folium.TileLayer(
    tiles=mapidVVdry_fil['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Det2020VV Filtered',
  ).add_to(mapcom)
folium.TileLayer(
    tiles=mapidVHwet_fil['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Wet2020VH Filtered',
  ).add_to(mapcom)
folium.TileLayer(
    tiles=mapidVHdry_fil['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Det2020VH Filtered',
  ).add_to(mapcom)

In [7]:
## Calculate difference between wet (flooded) and dry seasons
differenceVH = wet2020VH_filtered.divide(dry2020VH_filtered)
differenceVV = wet2020VV_filtered.divide(dry2020VV_filtered)

## Apply Threshold
DIFF_UPPER_THRESHOLD = 1.25;
differenceVH_thresholded = differenceVH.gt(DIFF_UPPER_THRESHOLD);
differenceVV_thresholded = differenceVV.gt(DIFF_UPPER_THRESHOLD);

In [8]:
vv_masked = differenceVV_thresholded.updateMask(differenceVV_thresholded)
vh_masked = differenceVH_thresholded.updateMask(differenceVH_thresholded)

mapid_vv  = vv_masked.getMapId({'palette': ['blue']})
mapid_vh  = vh_masked.getMapId({'palette': ['blue']})

map1 = folium.Map(location=[-71.82400911149269,-36.29123750152623],zoom_start=1)
folium.TileLayer(
    tiles=mapid_vv['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='flooded areas VV - Red',
  ).add_to(map1)
folium.TileLayer(
    tiles=mapid_vh['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='flooded areas VH - Red',
  ).add_to(map1)
mapcom.add_child(folium.LayerControl())
map.add_child(folium.LayerControl())
map1.add_child(folium.LayerControl())
map1